In [ ]:
!gdown --id 16OaU9XuXOonte_lAvYth51BaTQXAf7GK

Downloading...
From: https://drive.google.com/uc?id=16OaU9XuXOonte_lAvYth51BaTQXAf7GK
To: /content/Hashtaggenerator.zip
53.3MB [00:00, 143MB/s] 


In [ ]:
!unzip '/content/Hashtaggenerator.zip'

Archive:  /content/Hashtaggenerator.zip
   creating: Hashtag generator/.git/
   creating: Hashtag generator/.git/branches/
  inflating: Hashtag generator/.git/config  
  inflating: Hashtag generator/.git/description  
 extracting: Hashtag generator/.git/HEAD  
   creating: Hashtag generator/.git/hooks/
  inflating: Hashtag generator/.git/hooks/applypatch-msg.sample  
  inflating: Hashtag generator/.git/hooks/commit-msg.sample  
  inflating: Hashtag generator/.git/hooks/fsmonitor-watchman.sample  
  inflating: Hashtag generator/.git/hooks/post-update.sample  
  inflating: Hashtag generator/.git/hooks/pre-applypatch.sample  
  inflating: Hashtag generator/.git/hooks/pre-commit.sample  
  inflating: Hashtag generator/.git/hooks/prepare-commit-msg.sample  
  inflating: Hashtag generator/.git/hooks/pre-push.sample  
  inflating: Hashtag generator/.git/hooks/pre-rebase.sample  
  inflating: Hashtag generator/.git/hooks/pre-receive.sample  
  inflating: Hashtag generator/.git/hooks/update.sam

In [ ]:
!pip install flask-ngrok

In [ ]:
from flask import Flask, render_template, request
import tensorflow as tf
import cv2
from keras.models import load_model
import numpy as np
from keras.applications.resnet import ResNet50
from keras.optimizers import adam_v2
from keras.layers import Dense, Flatten,Input, Convolution2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector,Concatenate
from keras.models import Sequential, Model
from keras.utils import np_utils
from keras.preprocessing import image, sequence
import cv2
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from flask_ngrok import run_with_ngrok
import nltk
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import string
import re
import random
import requests
from bs4 import BeautifulSoup
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
from synonyms import Synonyms





vocab = np.load('/content/Hashtaggenerator/vocab.npy', allow_pickle=True)

vocab = vocab.item()

inv_vocab = {v:k for k,v in vocab.items()}


print("+"*50)
print("vocabulary loaded")


embedding_size = 128
vocab_size = len(vocab)
max_len = 40


image_model = Sequential()

image_model.add(Dense(embedding_size, input_shape=(2048,), activation='relu'))
image_model.add(RepeatVector(max_len))


language_model = Sequential()

language_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=max_len))
language_model.add(LSTM(256, return_sequences=True))
language_model.add(TimeDistributed(Dense(embedding_size)))

conca = Concatenate()([image_model.output, language_model.output])
x = LSTM(128, return_sequences=True)(conca)
x = LSTM(512, return_sequences=False)(x)
x = Dense(vocab_size)(x)
out = Activation('softmax')(x)
model = Model(inputs=[image_model.input, language_model.input], outputs = out)

model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

model.load_weights('/content/Hashtaggenerator/mine_model_weights.h5')

print("="*150)
print("MODEL LOADED")

resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling='avg')


#resnet = load_model('/content/image-captioning-keras-resnet/model.h5')

print("="*150)
print("RESNET MODEL LOADED")


app = Flask(__name__,template_folder="/content/Hashtaggenerator/templates",static_url_path='/content/Hashtaggenerator/static')
run_with_ngrok(app)

app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 1


@app.route('/')
def index():
    return render_template('index.html')

@app.route('/after', methods=['GET', 'POST'])
def after():

    global model, resnet, vocab, inv_vocab

    img = request.files['file1']

    img.save('/content/Hashtaggenerator/static/file.jpg')

    print("="*50)
    print("IMAGE SAVED")


    
    image = cv2.imread('/content/Hashtaggenerator/static/file.jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image = cv2.resize(image, (224,224))

    image = np.reshape(image, (1,224,224,3))

    
    
    incept = resnet.predict(image).reshape(1,2048)

    print("="*50)
    print("Predict Features")


    text_in = ['startofseq']

    final = ''

    print("="*50)
    print("GETING Captions")

    count = 0
    while tqdm(count < 20):

        count += 1

        encoded = []
        for i in text_in:
            encoded.append(vocab[i])

        padded = pad_sequences([encoded], maxlen=max_len, padding='post', truncating='post').reshape(1,max_len)

        sampled_index = np.argmax(model.predict([incept, padded]))

        sampled_word = inv_vocab[sampled_index]

        if sampled_word != 'endofseq':
            final = final + ' ' + sampled_word

        text_in.append(sampled_word)
    new_list=[]
    new_string=''
    meaning=[]
    newl=''
    nopunc = re.sub(r'[^\w\s]','', final)
    final=[word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    print(final)
    for word in final:
        new_list.append('#' + ''.join(word.split()))
        var= Synonyms(word)
        meaning=var.find_synonyms()
        print(meaning)
        for i in meaning:
          new_list.append('#'+i)
    random.shuffle(new_list)
    new_list=new_list[:40]
    new_string=''.join(new_list)
    print(new_string)
    return render_template('after.html', data=new_string) 
   
if __name__ == "__main__":
    app.run()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
++++++++++++++++++++++++++++++++++++++++++++++++++
vocabulary loaded
MODEL LOADED
RESNET MODEL LOADED
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c99d-35-230-108-27.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
